# Pour regrouper des achats par SCC (Strongly Connected Components) https://www.geeksforgeeks.org/strongly-connected-components/

Deniz Sert
12/06/23

In [6]:
from tqdm.notebook import tqdm as progressbar
import numpy as np
import pprint
from numba import jit, njit
import pandas as pd
import numpy as np
from collections import defaultdict

In [7]:
df_adjacency_list = pd.read_parquet('early_adj.parquet')

In [4]:
def cluster_nodes(adjacency_list):
    dag = create_directed_graph(adjacency_list)
    clusters = identify_clusters(dag)
    
    # Analyze clusters and identify specific characteristics
    
    return clusters

def create_directed_graph(adjacency_list):
    dag = defaultdict(list)
    for entry in adjacency_list:
        source = entry["addr_id1"]
        destination = entry["addr_id2"]
        value = entry["value"]
        height = entry["height"]
        dag[source].append((destination, value, height))
    
    return dag

def identify_clusters(dag):
    # Use Strongly Connected Components (SCC) algorithm
    # to identify clusters within the DAG
    clusters = []
    
    # Implement SCC algorithm here
    
    return clusters

## SCC peut été implementé par l’algorithme de Tarjan

In [9]:
def tarjans_algorithm(graph):
    index_counter = [0]
    stack = []
    lowlink = {}
    indices = {}
    visited = set()
    result = []

    with open('result.txt', 'w+') as f:
        def strongconnect(node):
            indices[node] = index_counter[0]
            lowlink[node] = index_counter[0]
            index_counter[0] += 1
            stack.append(node)
            visited.add(node)

            successors = list(graph.loc[graph['addr_id1'] == node]['addr_id2'])
            for successor in successors:
                if successor not in visited:
                    strongconnect(successor)
                    lowlink[node] = min(lowlink[node], lowlink[successor])
                elif successor in stack:
                    lowlink[node] = min(lowlink[node], indices[successor])

            if lowlink[node] == indices[node]:
                scc = []
                while True:
                    successor = stack.pop()
                    scc.append(successor)
                    if pd.isnull(successor) or successor == node:
                        break
                result.append(scc)
                f.write(str(scc) + '\n')

        for node in progressbar(graph['addr_id1'].unique()):
            if node not in visited:
                strongconnect(node)

        return result
    f.close()


strongly_connected_components = np.array(tarjans_algorithm(df_adjacency_list))
# print all the elements in the array whose length are greater than 1
print([x for x in strongly_connected_components if len(x) > 1])

  0%|          | 0/877912 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [8]:
from tqdm.notebook import tqdm as progressbar
import numpy as np
import pprint
from numba import jit, njit
import pandas as pd
import numpy as np
from collections import defaultdict

df_adjacency_list = pd.read_parquet('early_adj.parquet')
from tqdm.notebook import tqdm as progressbar
import numpy as np
import pprint
from numba import jit, njit
import pandas as pd
import numpy as np
from collections import defaultdict

df_adjacency_list = pd.read_parquet('early_adj.parquet')

def identify_clusters(dag):
    # Use Strongly Connected Components (SCC) algorithm
    # to identify clusters within the DAG
    clusters = []
    
    # Implement SCC algorithm here
    
    return clusters

def tarjans_algorithm(graph):
    index_counter = [0]
    stack = []
    lowlink = {}
    indices = {}
    on_stack = {}  # Track whether a node is on the stack
    result = []

    def strongconnect(node):
        indices[node] = index_counter[0]
        lowlink[node] = index_counter[0]
        index_counter[0] += 1
        stack.append(node)
        on_stack[node] = True

        successors = list(graph.loc[graph['addr_id1'] == node]['addr_id2'])
        for successor in successors:
            if successor not in indices:
                strongconnect(successor)
                lowlink[node] = min(lowlink[node], lowlink[successor])
            elif successor in on_stack:
                lowlink[node] = min(lowlink[node], indices[successor])

        if lowlink[node] == indices[node]:
            scc = []
            while True:
                successor = stack.pop()
                del on_stack[successor]
                scc.append(successor)
                if indices[successor] == lowlink[node]:
                    break
            result.append(scc)

    for node in progressbar(graph['addr_id1'].unique()):
        if node not in indices:
            strongconnect(node)

    return result

if __name__ == "__main__":
    strongly_connected_components = np.array(tarjans_algorithm(df_adjacency_list))
    # print all the elements in the array whose length is greater than 1
    print([x for x in strongly_connected_components if len(x) > 1])

def tarjans_algorithm(graph):
    index_counter = [0]
    stack = []
    lowlink = {}
    indices = {}
    on_stack = {}  # Track whether a node is on the stack
    result = []

    def strongconnect(node):
        indices[node] = index_counter[0]
        lowlink[node] = index_counter[0]
        index_counter[0] += 1
        stack.append(node)
        on_stack[node] = True

        successors = list(graph.loc[graph['addr_id1'] == node]['addr_id2'])
        for successor in successors:
            if successor not in indices:
                strongconnect(successor)
                lowlink[node] = min(lowlink[node], lowlink[successor])
            elif successor in on_stack:
                lowlink[node] = min(lowlink[node], indices[successor])

        if lowlink[node] == indices[node]:
            scc = []
            while True:
                successor = stack.pop()
                on_stack.pop(successor)
                scc.append(successor)
                if indices[successor] == lowlink[node]:
                    # print(indices[successor])
                    break
            result.append(scc)

    for node in progressbar(graph['addr_id1'].unique()):
        if node not in indices:
            strongconnect(node)

    return result

if __name__ == "__main__":
    strongly_connected_components = np.array(tarjans_algorithm(df_adjacency_list))
    # print all the elements in the array whose length are greater than 1
    print([x for x in strongly_connected_components if len(x) > 1])

  0%|          | 0/877912 [00:00<?, ?it/s]

KeyboardInterrupt: 